# **THE CODE WORKS ON  MY LOCAL MACHINE / LAPTOP NO SPECIAL ENVS**

# **Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Add Path To Current Directory**

In [2]:
cd /content/drive/MyDrive/Spring 2024/Reinforcement Learning /Project/FIFA 25/dm_control

/content/drive/MyDrive/Spring 2024/Reinforcement Learning /Project/FIFA 25/dm_control


# **Show What Is In The Current Folder**

In [3]:
ls

CODE.docx              fifa.py               soccer_training4.mp4
environment_names.txt  soccer_training2.mp4  soccer_training5.mp4
fifa_1..ipynb          soccer_training3.mp4  soccer_training.mp4


# **Pip Install The Libraries**

In [4]:
!pip install dm_control


# **Can Someone Search the Bellow Libraries**

!pip install dm_control[tf]

!pip install git+https://github.com/google-deepmind/dm_control.git

# **Initilize and Call the Modules**

In [5]:
import os
import numpy as np
from dm_control.locomotion import soccer as dm_soccer
from dm_control.locomotion.soccer import camera
from dm_control.locomotion.soccer import soccer_ball
import imageio


/usr/local/lib/python3.10/dist-packages/glfw/__init__.py:914: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


# **CPU OR GPU**

Instead of setting MUJOCO_GL to "glfw", which is used for software rendering, you can set it to "egl" or "gl", which are OpenGL backends optimized for GPU rendering.

EGL is a good choice if you're running the code on a server or in an environment without a display (headless).

GL is a suitable choice if you're running the code on a desktop or a workstation with a display

In [6]:
# Configure MuJoCo for software rendering (No GPU required)
os.environ["MUJOCO_GL"] = "glfw"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Explanation of the SoccerEnvironment Class**

The SoccerEnvironment class is designed to set up and simulate a soccer environment using the dm_control library. This class handles various tasks such as initializing the environment, setting up entities like soccer players and the soccer ball, and managing interactions between these entities during the simulation. The class provides several methods to control the simulation's behavior, step through the environment, and retrieve relevant data. Below is a detailed explanation of each section and method within the class.

#1. Initialization (__init__ Method)

The __init__ method is the constructor for the SoccerEnvironment class. It sets up the initial state of the soccer environment, defines its parameters, and initializes the entities involved in the simulation:

Environment Setup: The environment is initialized by calling dm_soccer.load() with specific parameters:

team_size=2: Configures the environment with two teams, each containing two players.

time_limit=10.0: Sets the time limit for each episode to 10 seconds.

disable_walker_contacts=False: Ensures that contact between walkers (players) is enabled.

enable_field_box=True: Enables the field boundaries.
terminate_on_goal=False: Keeps the simulation running even if a goal is scored.

walker_type=dm_soccer.WalkerType.HUMANOID: Specifies the type of player models used (in this case, humanoid).

Action Specifications: Retrieves the action specifications (self.env.action_spec()) which define the possible actions the agents (players) can take in the environment.

Environment Reset: Resets the environment to its initial state by calling self.env.reset(), which sets the initial timestep.

Soccer Ball Initialization: Creates a soccer ball entity using soccer_ball.regulation_soccer_ball(), which is a regulation-size soccer ball.

Adding Soccer Ball to Environment: Adds the soccer ball to the environment using self.env.task.arena.add_free_entity(self.soccer_ball).

Random State for Initialization: Creates a random state (random_state) to be used for initializing the environment.

Compile Ball with Environment Physics: Compiles the soccer ball with the environment's physics using self.soccer_ball.after_compile(self.env.physics, random_state).

Player Registration: Calls the register_players() method to register players (walkers) with the soccer ball.

#2. Registering Players (register_players Method)

The register_players method is responsible for associating the players (walkers) in the environment with the soccer ball, enabling them to interact with it.

Iterating Through Entities: Uses self.env.task.iter_entities() to iterate over all entities present in the environment.

Checking for Players: For each entity, checks if it is a player (walker) by looking for the walker_id attribute or ensuring the entity's class name starts with "walker".

Registering Players: Calls self.soccer_ball.register_player(entity) to register the identified players with the soccer ball.

#3. Stepping Through the Environment (step Method)

The step method advances the simulation by one timestep using the provided actions.

Executing Actions: Takes a list of actions as input and applies them to the environment using self.env.step(actions).

Updating Timestep: Updates the current state of the environment (self.timestep) with the new state after stepping through with the given actions.

Returning New Timestep: Returns the updated timestep, which contains the new state of the environment after the actions have been applied.

#4. Retrieving Positions (get_positions Method)

The get_positions method retrieves the positions of all entities, including players and the soccer ball, in the environment.

Extracting Positions: Uses a list comprehension to iterate over all body IDs (range(self.env.physics.model.nbody)) and retrieves their positions (self.env.physics.data.xpos[body_id]).

Returning Positions: Returns a list of positions representing the current locations of all entities in the environment.

#5. Getting Physics Object (get_physics Method)

The get_physics method provides access to the physics object of the environment.

Returning Physics Object: Returns the physics object (self.env.physics), which contains detailed information about the physical properties and dynamics of the environment.

In [7]:
class SoccerEnvironment:
    """Class to handle the soccer environment setup and simulation."""

    def __init__(self):
        # Initialize environment
        self.env = dm_soccer.load(
            team_size=2,
            time_limit=10.0,
            disable_walker_contacts=False,
            enable_field_box=True,
            terminate_on_goal=False,
            walker_type=dm_soccer.WalkerType.HUMANOID
        )
        self.action_specs = self.env.action_spec()
        self.timestep = self.env.reset()

        # Initialize the soccer ball
        self.soccer_ball = soccer_ball.regulation_soccer_ball()

        # Add soccer ball to the environment
        self.env.task.arena.add_free_entity(self.soccer_ball)

        # Create a random state for initialization
        random_state = np.random.RandomState(42)

        # Compile ball with environment physics
        self.soccer_ball.after_compile(self.env.physics, random_state)

        # Register players with the soccer ball
        self.register_players()

    def register_players(self):
        """Register players (walkers) with the soccer ball."""
        # Use `iter_entities()` to access all entities in the environment
        for entity in self.env.task.iter_entities():
            # Check if the entity is a player (walker) using its type or attributes
            if hasattr(entity, 'walker_id') or entity.__class__.__name__.lower().startswith("walker"):
                self.soccer_ball.register_player(entity)

    def step(self, actions):
        """Step through the environment with given actions."""
        self.timestep = self.env.step(actions)
        return self.timestep

    def get_positions(self):
        """Get positions of entities (players and ball) in the environment."""
        positions = [self.env.physics.data.xpos[body_id] for body_id in range(self.env.physics.model.nbody)]
        return positions

    def get_physics(self):
        """Get the physics object from the environment."""
        return self.env.physics

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Explanation of the MultiplayerCamera Class**

The MultiplayerCamera class is designed to manage the camera settings and behavior for visualizing a multiplayer soccer environment using the dm_control library. This class configures the camera to follow and track the entities within the environment, ensuring that the camera provides a clear view of the ongoing game. It includes methods to initialize the camera with the environment's physics, update its position after each simulation step, and render frames for visualization. Below is a detailed breakdown of each component and method within the class.

#1. Initialization (__init__ Method)

The __init__ method serves as the constructor for the MultiplayerCamera class. It sets up the camera with specific parameters to define its behavior and appearance:

Camera Configuration: The camera is initialized using camera.MultiplayerTrackingCamera() with the following parameters:

min_distance=3.0: Sets the minimum distance between the camera and the tracked entities to avoid collision or overly close views.

distance_factor=1.5: Defines a scaling factor for the camera's distance, controlling how far the camera will be from the entities based on their spread.

smoothing_update_speed=0.1: Controls the speed at which the camera smoothly follows the entities, preventing abrupt movements.

azimuth=90: Sets the camera's horizontal angle relative to the scene (measured in degrees).

elevation=-30: Sets the camera's vertical angle relative to the scene (measured in degrees).

width=640: Specifies the width of the camera's rendered output in pixels.

height=480: Specifies the height of the camera's rendered output in pixels.

These parameters help create a dynamic, smooth, and adjustable camera view that provides a comprehensive visualization of the soccer environment.

#2. Initializing the Camera (initialize Method)

The initialize method sets up the camera using the environment's physics object. This is essential to align the camera with the environment's current state:

Compiling with Physics: The camera is compiled with the environment's physics object by calling self.camera.after_compile(physics). This ensures that the camera has access to the necessary data for accurate tracking and rendering.


Getting Initial Positions: A list comprehension is used to retrieve the initial positions of all entities (body_id) in the environment from the physics data (physics.data.xpos[body_id]).


Initializing Camera Episode: The camera is initialized to begin tracking the episode using the initial positions (self.camera.initialize_episode(initial_positions)). This sets the camera's focus and ensures it is correctly positioned from the start of the simulation.

#3. Updating the Camera (update Method)

The update method refreshes the camera's position and orientation after each step in the simulation.

Updating After Each Step: The camera's position and tracking are updated using self.camera.after_step(positions), which takes the current positions of entities as input. This ensures that the camera smoothly follows the movement of entities during the game, maintaining a clear view.

#4. Rendering the Frame (render Method)

The render method generates a visual representation of the current state of the environment from the camera's perspective.

Rendering the Frame: The method calls self.camera.render() to produce a frame that captures the current view of the environment. This frame can be displayed in real-time or recorded for later analysis.

In [8]:
class MultiplayerCamera:
    """Class to handle the camera settings and behavior."""

    def __init__(self):
        self.camera = camera.MultiplayerTrackingCamera(
            min_distance=3.0,
            distance_factor=1.5,
            smoothing_update_speed=0.1,
            azimuth=90,
            elevation=-30,
            width=640,
            height=480
        )

    def initialize(self, physics):
        """Initialize the camera with the environment's physics."""
        self.camera.after_compile(physics)
        initial_positions = [physics.data.xpos[body_id] for body_id in range(physics.model.nbody)]
        self.camera.initialize_episode(initial_positions)

    def update(self, positions):
        """Update the camera after each step."""
        self.camera.after_step(positions)

    def render(self):
        """Render the current frame using the tracking camera."""
        return self.camera.render()



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Explanation of the SoccerTraining Class**

The SoccerTraining class is responsible for handling the entire training process of a soccer simulation, including reward calculations, player actions, and video recording of the session. This class integrates the previously defined SoccerEnvironment and MultiplayerCamera classes to create a cohesive system for training and evaluating players in a soccer game environment. Below is a detailed breakdown of each component and method within the class.

#1. Initialization (__init__ Method)

The __init__ method serves as the constructor for the SoccerTraining class. It sets up the necessary components to start and manage the training process:

Environment Setup: The environment is initialized by creating an instance of the SoccerEnvironment class (self.env), which sets up the soccer field, players, and ball.

Camera Setup: A camera is initialized through an instance of the MultiplayerCamera class (self.camera) to visualize the game.

Video Recording Configuration: The video_writer is configured to record the training session as an MP4 video file named "soccer_training5.mp4" at a frame rate of 30 frames per second.

Frame Counter: The frame_count variable is initialized to 0 to keep track of the number of frames processed during the training session.

#2. Calculating Distance to Ball (calculate_distance_to_ball Method)

The calculate_distance_to_ball method computes the Euclidean distance between a player and the ball, which is essential for determining rewards based on player behavior:

Distance Calculation: The method takes two arguments, player_position and ball_position, which represent the positions of a player and the ball, respectively. It uses NumPy's np.linalg.norm() to compute the Euclidean distance between these two points.

#3. Calculating Rewards (calculate_rewards Method)

The calculate_rewards method calculates the rewards for each player based on their behavior in the environment:

Initializing Rewards and Physics: An empty rewards list is initialized to store the rewards for each player. The environment's physics object is retrieved using self.env.get_physics().

Accessing Ball Position: The position of the soccer ball is obtained using its geometry name ('soccer_ball/geom'). If the ball's position cannot be accessed, an exception is caught, and an error message is printed.

Player Root Names: The method defines a list of player root names (player_root_names) that match the exact entity names within the environment: ['home0/root', 'home1/root', 'away0/root', 'away1/root'].

Loop Through Players: For each player, the method:

Retrieves the player's position using physics.named.data.xpos[player_name].
Calculates the distance between the player and the ball.

Computes rewards for getting closer to the ball (distance_reward), kicking the ball (kick_reward), and moving (movement_reward).

Sums the rewards to calculate the total reward for the player and appends it to the rewards list.
Handles any errors in accessing player data gracefully using exception handling.

#4. Printing Rewards (print_rewards Method)

The print_rewards method outputs the rewards for each player to the console:

Output Formatting: The method iterates through the rewards list and prints each player's reward in a formatted manner (e.g., "Player 0 reward: 1.23").

#5. Running the Training Session (run_training Method)

The run_training method executes the main training loop, managing the interactions between the players and the environment:

Camera Initialization: The camera is initialized with the physics object to start tracking the simulation.

Main Training Loop: While the environment has not reached its final timestep:

Generate Random Actions: Random actions are generated for all players using their action specifications (self.env.action_specs).

Step through the Environment: The simulation advances by one step using self.env.step(actions).

Update Camera and Calculate Rewards: The camera is updated with the current positions of the entities, and rewards are calculated based on player behaviors.
Print Rewards and Observations: Rewards and observations are printed for each player.

Record Frames: Frames are recorded at regular intervals (every 5th frame) using the self.camera.render() method.

Increment Frame Counter: The frame_count is incremented after each loop iteration.
Video Closing and Completion: Once the training session is complete, the video writer is closed, and a message is printed indicating the successful completion of the training.

In [9]:
class SoccerTraining:
    """Class to handle the training process, rewards, and recording."""

    def __init__(self):
        self.env = SoccerEnvironment()
        self.camera = MultiplayerCamera()
        self.video_writer = imageio.get_writer("soccer_training6.mp4", fps=30)
        self.frame_count = 0

    def calculate_distance_to_ball(self, player_position, ball_position):
        """Calculate the Euclidean distance between the player and the ball."""
        return np.linalg.norm(np.array(player_position) - np.array(ball_position))

    def calculate_rewards(self):
        """Calculate rewards based on player behaviors."""
        rewards = []
        physics = self.env.get_physics()

        # Access the ball position using the correct name
        ball_geom_name = 'soccer_ball/geom'  # Correct soccer ball geometry name
        try:
            ball_geom_id = physics.model.name2id(ball_geom_name, 'geom')
            ball_position = physics.data.xpos[ball_geom_id]
        except Exception as e:
            print(f"Error retrieving ball position: {e}")
            return rewards

        # List of player root names based on the environment
        player_root_names = ['home0/root', 'home1/root', 'away0/root', 'away1/root']  # Updated to match exact names

        for player_name in player_root_names:
            try:
                # Access the root body's position
                player_position = physics.named.data.xpos[player_name]

                # Calculate the distance to the soccer ball
                distance_to_ball = self.calculate_distance_to_ball(player_position, ball_position)

                # Reward for getting closer to the ball
                distance_reward = max(0, 1.0 - distance_to_ball)

                # Reward for kicking the ball
                kick_reward = 2.0 if self.env.soccer_ball.hit else 0

                # Access the linear velocity of the root body
                velocity = np.linalg.norm(physics.named.data.qvel[player_name])

                # Reward for moving (learning to walk/run)
                movement_reward = min(velocity, 1.0)

                # Total reward
                total_reward = distance_reward + kick_reward + movement_reward
                rewards.append(total_reward)

            except KeyError as e:
                print(f"Key error while accessing data for player '{player_name}': {e}")
                continue

        return rewards

    def print_rewards(self, rewards):
        """Print the rewards for each player."""
        for i, reward in enumerate(rewards):
            print(f"Player {i} reward: {reward:.2f}")

    def run_training(self):
        """Run the soccer training session."""
        self.camera.initialize(self.env.get_physics())

        while not self.env.timestep.last():
            # Generate random actions for all players
            actions = [
                np.random.uniform(action_spec.minimum, action_spec.maximum, size=action_spec.shape)
                for action_spec in self.env.action_specs
            ]

            # Step through the environment
            self.env.step(actions)

            # Update camera and calculate rewards
            entity_positions = self.env.get_positions()
            self.camera.update(entity_positions)
            rewards = self.calculate_rewards()

            # Print rewards and observations
            self.print_rewards(rewards)
            for i in range(len(self.env.action_specs)):
                print(f"Player {i}: observations = {self.env.timestep.observation[i]}")

            # Record frames at regular intervals
            if self.frame_count % 5 == 0:
                frame = self.camera.render()
                self.video_writer.append_data(frame)

            self.frame_count += 1

        self.video_writer.close()
        print("Training completed and video saved as soccer_training6.mp4.")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
if __name__ == "__main__":
    # Run the soccer training
    training = SoccerTraining()
    training.run_training()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/glfw/__init__.py:914: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


FatalError: an OpenGL platform library has not been loaded into this process, this most likely means that a valid OpenGL context has not been created before mjr_makeContext was called

# **RUN LOCALLY IGNORE THIS ERROR COLAB HAS NO SUPPORT FOR IT**